In this notebook we evaluate performance of pretrained Gemma model.

# PREREQUISITES:

In [1]:
from transformers import GemmaForSequenceClassification
import torch
import pandas as pd
import torch.nn.functional as F
from huggingface_hub import login
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import GemmaTokenizerFast
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os

In [8]:
login()

# Load dataset

In [3]:
current_directory = os.getcwd() # get current directory
file_path = os.path.join(current_directory, '..', '..', 'data', 'processed', 'clean_data.csv') # navigate to folder with preprocessed data
data = pd.read_csv(file_path)
print(data.head(5))

                                        heading_text  bias_rating
0  chicago gun violence spikes and increasingly f...            0
1  'bullets just came from nowhere': fourth of ju...            1
2  dozens of shootings across us mark bloody july...            2
3  federal government will run out of cash on oct...            2
4  yellen tells congress that u.s. will run out o...            0


# Split and tokenize

In [11]:
X = data['heading_text'].values
y = data['bias_rating'].values

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained('google/gemma-2b')
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, max_length=512)

In [14]:
class NewsDataset(torch.utils.data.Dataset):
    """
    We need this class to manage and load
    encoded inputs and corresponding labels into a format compatible
    with PyTorch's DataLoader for model training and evaluation :)
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, y_train)
test_dataset = NewsDataset(test_encodings, y_test)

In [15]:
device = 0 if torch.cuda.is_available() else -1

# Load the model and test it

In [ ]:
model_id  = "google/gemma-2b-it"


model = GemmaForSequenceClassification.from_pretrained(
    model_id, 
    num_labels = 3,
    ).to(device)

In [27]:
torch.cuda.empty_cache()

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    
    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [36]:
training_args = TrainingArguments(
    output_dir="./results_baseline",     
    per_device_eval_batch_size=16,   
    logging_dir="./logs_baseline",       
    do_train=False,                  # Disable training
    do_eval=True,                    
    evaluation_strategy="epoch",    
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

/home1/s5153484/venvs/my_env/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


# Evaluate model

In [37]:
eval_results = trainer.evaluate()

# Print baseline performance
print("Baseline performance:", eval_results)

Baseline performance: {'eval_loss': 1.9030852317810059, 'eval_model_preparation_time': 0.0058, 'eval_accuracy': 0.3770981834904576, 'eval_precision': 0.3860343863973383, 'eval_recall': 0.3770981834904576, 'eval_f1': 0.3110451359867519, 'eval_runtime': 227.0357, 'eval_samples_per_second': 19.156, 'eval_steps_per_second': 1.198}
